<a href="https://colab.research.google.com/github/maleehahassan/HIDA_Into_to_DL/blob/main/14_feedforward_squirrels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from tensorflow import keras
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix

sns.set_theme(style='white')

In [ ]:
# %matplotlib inline

First, download a dataset about squirrel sightings in New York City.

In [ ]:
# simply using pandas
url = 'https://data.cityofnewyork.us/resource/vfnx-vebw.csv?$limit=5000'
df = pd.read_csv(url)
df.shape

In [ ]:
df.head()

In [ ]:
df.describe(include='all')

## Exploratory data analysis
Inspect the dataset some more, using in particular `dtypes` or `describe`. Find out how often the different behaviours were observed!

In [ ]:
# using dtypes

In [ ]:
# using pandas describe

If you want, you can draw a `pairplot` – but it will be very big!

Draw sighting locations using `sns.relplot` or `sns.jointplot`, then explore the distributions of a few interesting features and the `hue` parameter.

In [ ]:
# the primary_fur_color is probably interesting
sns.relplot(data=df,
            ...,
            )

In [ ]:
# or the shift


In [ ]:
# a jointplot also shows the marginal distributions
sns.jointplot(
    ...
              )

## Feature Cleaning / Encoding
This dataset definitely needs some cleaning. It has a mix of categorical, boolean and numerical values; a lot of `NA`s and some string columns with rare annotation.

In [ ]:
def prepare_dataset(df: pd.DataFrame,
                    target: str = 'primary_fur_color',
                    ) -> tuple[np.array, np.array, OneHotEncoder]:
    # make a copy so we don't modify the original dataframe
    df = df.copy()

    if target not in df.columns:
        raise ValueError(f'column {target} not found in dataset!')

    # drop rows where the target column isn't defined
    df = df.dropna(subset=[target])
    print(f'{len(df)} rows remaining')

    # not all columns can be used as training features.
    # we also treat categoricals and booleans differently.
    cat_cols = ['age', 'hectare', 'shift']
    bool_cols = [
        'running', 'chasing', 'climbing', 'eating', 'foraging',
        'kuks', 'quaas', 'moans', 'tail_flags', 'tail_twitches',
        'approaches', 'indifferent', 'runs_from',
    ]

    ok_features = [c for c in [
        'hectare', 'shift', 'primary_fur_color',
        'highlight_fur_color', 'specific_location',
        'running', 'chasing', 'climbing', 'eating',
        'foraging', 'kuks', 'quaas', 'moans',
        'tail_flags', 'tail_twitches', 'approaches',
        'indifferent', 'runs_from',
    ] if c in df.columns]

    # Keep only available columns
    cat_cols = [c for c in cat_cols if c in df.columns and c != target]
    bool_cols = [c for c in bool_cols if c in df.columns and c != target]

    # fill booleans with False, then convert to int
    df[bool_cols] = df[bool_cols].fillna(False).astype(int)

    # encoder for categorical features
    preprocessor = ColumnTransformer(transformers=[
        ('bools', 'passthrough', bool_cols),
        ('cats', OneHotEncoder(handle_unknown='ignore'), cat_cols)
    ])

    # encode and transform features
    X = preprocessor.fit_transform(df)

    # encode the target column
    y_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    y = y_encoder.fit_transform(df[[target]])

    return X, y, y_encoder


In [ ]:
X, y, y_encoder = prepare_dataset(df, target='primary_fur_color')  # 'primary_fur_color' 'shift', 'age'

In [ ]:
# finally make a train-test-split
X_train, X_test, y_train, y_test = train_test_split(
    ...)

print(f'train: {X_train.shape}\ntest:  {X_test.shape}')
print('Classes:', y_encoder.categories_[0])

### Reproducibility

In [ ]:
from numpy.random import seed
from tensorflow.random import set_seed
seed(1)
set_seed(2)
sns.set_theme(style='white')

## Predict Squirrel Fur Color

In [ ]:
# this function sets up the model. You can make it as complex or simple as you want. In particular, feel free to try dropout, or try different sizes.
def make_multiclass_model(input_shape, n_classes) -> keras.Model:
    ...

# once your function is finished, you can build your model:
model = make_multiclass_model((X_train.shape[1],), y_train.shape[1])
# now compile it with the correct loss and optimizer:
model.compile(
    ...
              )
model.summary()

In [ ]:
# train the model
history = model.fit(...,
                    validation_split=.2,
                    verbose=2)

y_train_predict = model.predict(X_train)
y_test_predict = model.predict(X_test)

In [ ]:
# turn the history into a DataFrame for easier analysis
history_df = pd.DataFrame.from_dict(history.history)
history_df.head()

In [ ]:
# plot the training and validation loss and accuracy
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(6, 3))

sns.lineplot(..., ax=ax0)
ax0.set_xlabel('epochs')
sns.lineplot(..., ax=ax1)
plt.tight_layout()


In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
f'test loss: {test_loss:.2f}, test accuracy {test_accuracy:.2f}'

In [ ]:
y_pred = model.predict(X_test)
prediction = y_encoder.inverse_transform(y_pred)
ground_truth = y_encoder.inverse_transform(y_test)

In [ ]:
cm = confusion_matrix(ground_truth, prediction)
print(cm)

In [ ]:
# To confirm which axis is the truth and which is the prediction
np.unique(ground_truth, return_counts=True)

In [ ]:
ax = sns.heatmap(cm, annot=True, square=True,
            yticklabels=y_encoder.categories_[0],
            xticklabels=y_encoder.categories_[0],
            )
ax.set(xlabel='prediction', ylabel='ground truth')

## Extra Task: Predict `shift`
This means whether the squirrel was seen before or after noon.

## Extra Task: Feature Engineering
Build a feature that indicates whether a squirrel will accept and eat a nut from you:
It `approaches` or `forages`, or `eating` already. Also, it is not `indifferent` or `runs_from` you. Then build a predictor for this feature!

## Extra Task: Regression
Build a synthetic feature that encodes how chatty a squirrel is. For that, combine `kuks`, `moans` and `quaas`.